In [ ]:
import json
import re
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_path = r'../../data/ots_snowfall_data.json'

In [44]:
with open(data_path, 'r') as json_file:
    json_file_read = json_file.read()
    json_file_clean = json.loads(json_file_read)
    result_df = pd.DataFrame(json_file_clean)
result_df.tail()

,station,what_data,year,url,data
5431,Bruce Mound,base,2018,https://www.onthesnow.com/wisconsin/bruce-moun...,"{'Dec 2018': {'15': 12, '16': 12, '17': 12, '1..."
5432,Sleeping Giant,base,2018,https://www.onthesnow.com/wyoming/sleeping-gia...,"{'Dec 2018': {'14': 30, '15': 28, '16': 26, '1..."
5433,Elko SnoBowl,base,2018,https://www.onthesnow.com/nevada/elko-snobowl/...,{}
5434,Eagle Point,base,2018,https://www.onthesnow.com/utah/eagle-point/his...,"{'Dec 2018': {'28': 28, '29': 28, '30': 28, '3..."
5435,Pine Knob,base,2018,https://www.onthesnow.com/michigan/pine-knob-s...,"{'Nov 2018': {'30': 24}, 'Dec 2018': {'1': 24,..."


In [93]:
def data_exploder(df, station):
    """create dataframe of station data in tabular form"""
    station_df = (df.query('station == @station')[['data', 'what_data']])
    def cleaner(df, data_pt):
        """pull data out of json into columns"""
        return (pd.json_normalize(df.loc[df.what_data==data_pt, 'data'])
                .sum()
                .T.to_frame()
                .rename(columns={0: data_pt}))
    base_df = cleaner(station_df, 'base')
    snowfall_df = cleaner(station_df, 'snowfall')
    station_data = (pd.concat([base_df, snowfall_df], axis=1)
                   )
    dt_format = "%b %Y.%d"
    station_data.index = pd.to_datetime(station_data.index, format=dt_format)
    return station_data
result_df.pipe(data_exploder, 'Crystal Mountain')

,base,snowfall
2010-11-16,14.0,NaN
2010-11-17,14.0,NaN
2010-11-18,35.0,16.0
2010-11-20,28.0,6.0
2010-11-23,34.0,NaN
...,...,...
2019-04-19,94.0,NaN
2019-04-20,90.0,NaN
2019-04-21,92.0,NaN
2014-11-22,NaN,2.0


In [176]:
def annualizer(df):
    df.index.name = 'timestamp'
    def day_standardizer(day):
        return (day+100)%366
    df_by_yr = (df
                .reset_index()
                .assign(year=lambda x: x.timestamp.dt.year)
                .assign(month=lambda x: x.timestamp.dt.month) 
                .assign(day=lambda x: x.timestamp.dt.day) 
                .assign(dayofyr=lambda x: day_standardizer(x.timestamp.dt.dayofyear))
                .reset_index()
                .set_index('dayofyr')
                .loc[:, ['base', 'snowfall']]
                .groupby(['dayofyr'])
                .agg([np.mean, 'std'])
                .reset_index()
               )
    df_by_yr.columns = df_by_yr.columns.map('_'.join)
    return df_by_yr

In [179]:
one_station = (result_df
               .pipe(data_exploder, 'Arapahoe Basin')
               .pipe(annualizer))
one_station

,dayofyr_,base_mean,base_std,snowfall_mean,snowfall_std
0,10,10.0,NaN,1.0,NaN
1,11,10.0,NaN,1.0,NaN
2,12,10.0,NaN,NaN,NaN
3,18,10.0,NaN,NaN,NaN
4,20,18.0,0.0,NaN,NaN
...,...,...,...,...,...
260,279,28.0,NaN,NaN,NaN
261,280,28.0,NaN,NaN,NaN
262,281,19.0,NaN,NaN,NaN
263,282,19.0,NaN,NaN,NaN


In [180]:
base_chart = (alt.Chart(one_station)
              .mark_line().encode(x='dayofyr_:Q',
                                  y='base_mean:Q',
                                  tooltip=['base_mean:Q']
                                  )
              )
base_chart

alt.Chart(...)

In [59]:
pd.plotting.autocorrelation_plot(df["R"].resample("1y").median())

array([  6.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,
        28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,
        39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,
        50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
        61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
        72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,
        83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,
        94.,  95.,  96.,  97.,  98.,  99., 100., 101., 102., 103., 104.,
       107., 108., 110., 421.])